# Computing the sets $\Omega(I_{2,n})$

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

We briefly recall the definitions of $I_w$ and $\Omega(I)$. Let $\Delta = \mathcal{A}(I)$ be the point configuration of $I$. A vector $\mathsf{w}\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{\mathsf{w}}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{\mathsf{w}}(\Delta)\}$$

and the ideal they generate is

$$I_{\mathsf{w}} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

For an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{\mathsf{w} \in \mathbb{R}^{n} \, : \, \mathrm{in}_{\mathsf{w}} I = I_{\mathsf{w}} \}. $$

This set is the support of a subfan of the secondary fan $\mathrm{Sec}(\Delta)$.

Further, recall that the ideal $I_{r,n}$ denotes the Pluecker ideal generating the Grassmannian $Gr(r,n)$.

---

We find the set $\Omega(I_{2,n})$ for $n=4,5,6$ by computing the underlying subfan of the secondary fan. 

In [ ]:
using Oscar;
using Combinatorics;
pm = Polymake;
include("InitialIdealsRegularSubdivisions.jl");

We see that $\Omega(I_{2,4})$ is the entire space, i.e., $(I_{2,4})_w=\mathrm{in}_w I_{2,4}$ for all $w$.

In [ ]:
Plu24, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4], 2));
I24 = grassmann_pluecker_ideal(Plu24, 2, 4);
Delta24 = hypersimplex(2,4);
Delta24_polymake = pm.polytope.hypersimplex(2,4);
Sec24 = polyhedral_fan(pm.fan.secondary_fan(Delta24_polymake));
F = Omega_fan(I24, Delta24, Sec24)
is_complete(F)

The set $\Omega(I_{2,5})$ is the union of 72 of the 102 maximal secondary cones of $\Delta(2,5)$.

In [ ]:
Plu25, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4,5], 2));
I25 = grassmann_pluecker_ideal(Plu25, 2, 5);
Delta25 = hypersimplex(2,5);

Delta25_polymake = Polymake.polytope.hypersimplex(2,5);
Sec25 = polyhedral_fan(Polymake.fan.secondary_fan(Delta25_polymake));
Omega25 = Omega_fan(I25, Delta25, Sec25);
println(n_maximal_cones(Omega25), " ", n_maximal_cones(Sec25))

For $I_{2,6}$ we use precomputed secondary fan data in MaRDI file format. To optimize, we first check which of the 1235 secondary rays are contained in $\Omega(I_{2,6})$. This allows us to process only those cones which are generated by such rays. As a result, we see that 21240 of th 194160 maximal secondary cones lie in $\Omega(I_{2,6})$. This computation takes  several hours.

In [ ]:
I26 = grassmann_pluecker_ideal(2, 6);
Sec26 = load("precomputed_data/secondary_fan_D26.fan")
good_rays = Bool[]
for r in eachrow(Sec26.pm_fan.RAYS)
    w = Int.(lcm(denominator.(r))*r)
    inI = initial(I26, tropical_semiring_map(QQ), w)
    Iw = ideal_w(I26, w)
    push!(good_rays, Iw == inI)
end
sum(good_rays)

In [ ]:
MC = reshape(Sec26.pm_fan.MAXIMAL_CONES, 194160, :)
c = 0
for i in 1 : 194160
    if prod(MC[i, :] .<= good_rays) == 1
        w = sum(R[j, :] for j in MC[i, :].s)
        w = Int.(lcm(denominator.(w))*w)
        inI = initial(I26, nu_t, w)
        Iw = ideal_w(I26, w)
        c += inI == Iw
    end
end
c